# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
# print(geoapify_key)

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,14.15,90,100,8.97,SH,1722305263
1,1,nar'yan-mar,67.6713,53.0870,11.82,75,100,5.77,RU,1722305264
2,2,albany,42.6001,-73.9662,22.24,88,39,2.22,US,1722305259
3,3,rikaze,29.2500,88.8833,16.83,29,0,0.77,CN,1722305267
4,4,carnarvon,-24.8667,113.6333,19.28,69,0,4.07,AU,1722305268


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
city_data_df.count()

City_ID       554
City          554
Lat           554
Lng           554
Max Temp      554
Humidity      554
Cloudiness    554
Wind Speed    554
Country       548
Date          554
dtype: int64

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
trim_city_data_df = city_data_df[ 
    (city_data_df["Max Temp"] < 36) & (city_data_df["Max Temp"] > 21) 
    & (city_data_df["Wind Speed"] < 5) 
    & (city_data_df["Cloudiness"] == 0) ]
# trim_city_data_df.count()

In [7]:
# Drop any rows with null values
trim_city_data_df = trim_city_data_df.dropna()
# trim_city_data_df.count()

In [8]:
# Display sample data
trim_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,bethel,41.3712,-73.4140,22.23,89,0,0.00,US,1722305283
26,26,kaspiyskiy,45.3929,47.3707,23.55,65,0,3.86,RU,1722305303
43,43,shetpe,44.1667,52.1167,22.40,43,0,1.22,KZ,1722305326
52,52,cape canaveral,28.4058,-80.6048,28.93,81,0,2.57,US,1722305338
73,73,ribeira brava,32.6500,-17.0667,23.34,79,0,1.01,PT,1722305368


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = trim_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
14,bethel,US,41.3712,-73.4140,89,
26,kaspiyskiy,RU,45.3929,47.3707,65,
43,shetpe,KZ,44.1667,52.1167,43,
52,cape canaveral,US,28.4058,-80.6048,81,
73,ribeira brava,PT,32.6500,-17.0667,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
kaspiyskiy - nearest hotel: Гостевой двор
shetpe - nearest hotel: Балумит
cape canaveral - nearest hotel: Holiday Inn Club Vacations Cape Canaveral Beach Resort
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
ouddorp - nearest hotel: Akershoek
gremda - nearest hotel: Business Hotel Sfax
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
vila velha - nearest hotel: Hotel Prainha
jiujiang - nearest hotel: 九江信华建国酒店
logan - nearest hotel: Hampton Inn & Suites Logan
sidi ech chahmi - nearest hotel: Vendom Khaled
surman - nearest hotel: فندق البرج
north bay - nearest hotel: Days Inn
garden city - nearest hotel: Quality Inn & Suites
newman - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
sultanah - nearest hotel: فندق جلنار
argostolion - nearest hotel: Ainos Hotel
padang - nearest hotel: losmen Surya
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
sirjan - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
14,bethel,US,41.3712,-73.4140,89,Hampton Inn Danbury
26,kaspiyskiy,RU,45.3929,47.3707,65,Гостевой двор
43,shetpe,KZ,44.1667,52.1167,43,Балумит
52,cape canaveral,US,28.4058,-80.6048,81,Holiday Inn Club Vacations Cape Canaveral Beac...
73,ribeira brava,PT,32.6500,-17.0667,79,Cheerfulway Bravamar Hotel
104,ouddorp,NL,51.8117,3.9347,70,Akershoek
116,gremda,TN,34.7500,10.7833,47,Business Hotel Sfax
118,birjand,IR,32.8663,59.2211,40,مهمانسرای جهانگردی بیرجند
136,vila velha,BR,-20.3297,-40.2925,87,Hotel Prainha
147,jiujiang,CN,29.7333,115.9833,63,九江信华建国酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)